<a href="https://colab.research.google.com/github/ganbozza/test/blob/main/Mega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !sudo apt-get update && sudo apt-get install megacmd
#!wget https://mega.nz/linux/repo/Arch_Extra/x86_64/megacmd-x86_64.pkg.tar.zst && sudo pacman -U "$PWD/megacmd-x86_64.pkg.tar.zst"
!wget https://mega.nz/linux/repo/xUbuntu_22.04/amd64/megacmd-xUbuntu_22.04_amd64.deb && sudo apt install "$PWD/megacmd-xUbuntu_22.04_amd64.deb"

In [ ]:
# @markdown <center><h3>Enter your MEGA email & password below</h3></center><br>
from functools import wraps
import errno
import signal


class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator
# INPUT YOUR MEGA ID
EMAIL = "ganbozza@gmail.com"  # @param {type:"string"}
PASSWORD = "alemega@2020"  # @param {type:"string"}
if not (EMAIL == "" or PASSWORD == ""):
    try:
        !mega-login {EMAIL} {PASSWORD}
    except TimeoutError:
        !mega-whoami
else:
    print("Please enter your MEGA email and password.")


Fetching nodes ||##########################################||(1/1 KB: 100.00 %)  
